In [ ]:
from summary_string import summary_string
from PIL import Image, ImageDraw
from macnet import Net
import torch
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model = Net(3).to(device)
data, meta_data = summary_string(model, input_size=(2, 96, 96))

In [ ]:
blk = (0,0,0)
clr = (255,100,0)
clr2 = (100,254,0)

In [ ]:
import math
"""
def make_layer(origin, size, t, angle=45):
    res = np.zeros((3,4,2))
    x,y = origin
    scos = size*math.cos(angle*math.pi/180)
    ssin = size*math.sin(angle*math.pi/180)
    res = []
    # rect
    res.append([(x,y), (x+t,y), (x+t,y+size), (x, y+size)])
    
    # top trap
    res.append([(x,y), (x+t,y), (x+t+ssin,y-scos), (x+ssin, y-scos)])

    # right trap
    res.append([(x+t,y), (x+t,y+size), (x+t+ssin, y+size-scos), (x+t+ssin,y-scos)])
    return res
"""
def iso_proj(xyz, a, b):
    cosb = math.cos(b*math.pi/180)
    cosa = math.cos(a*math.pi/180)
    sina = math.sin(a*math.pi/180)
    sinb = math.sin(b*math.pi/180)
    
    matrix = np.array([[cosb, 0, -sinb],
                       [sina*sinb, cosa, sina*math.cos(cosb)],
                       [cosa*sinb, -sina, cosa*math.cos(cosb)]])
    return tuple(np.matmul(matrix, xyz)[:2])

def make_layer(origin, sz, t, a=45, b=45):
    # TODO: convert to matrix multiplication
    shapes = []
    s = sz/2
    #shapes.append([(-s,-s,-s), (-s, s,-s), ( s, s,-s), ( s,-s,-s)])
    #shapes.append([(-s,-s,-s), (-s, s,-s), (-s, s, s), (-s,-s, s)])
    shapes.append([(-t,-s,-s), ( t,-s,-s), ( t,-s, s), (-t,-s, s)])
    
    shapes.append([( t, s, s), ( t, s,-s), ( t,-s,-s), ( t,-s, s)])
    #shapes.append([( s, s, s), (-s, s, s), (-s, s,-s), ( s, s,-s)])
    shapes.append([( t, s, s), (-t, s, s), (-t,-s, s), ( t,-s, s)])
    for i in range(len(shapes)):
        for j in range(len(shapes[i])):
            x,y=iso_proj(shapes[i][j],a,b)
            x = int(round(x))
            y = int(round(y))
            shapes[i][j] = (x+origin[0],y+origin[1])

    return shapes

In [ ]:


blk = (0,0,0)
clr = (255,100,0)
clr2 = (100,254,0)

im =  Image.new("RGB", (900,400), color=(255,255,255))
draw = ImageDraw.Draw(im)
layers = []
layers.append(make_layer((100,100), 150, 5,b=15))
spacing = iso_proj((20,0,0), 45, 15)
spacing = (100+spacing[0],100+spacing[1])
layers.append(make_layer(spacing, 150, 5,b=15))
for layer in layers:
    for shp in layer:
        draw.polygon(shp, fill=clr, outline=blk, width=1)
display(im)